In [1]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchRun

In [2]:
from langchain.llms import Ollama
ollama_llm = Ollama(model="openhermes")
# ollama_llm = Ollama(model="phi")

In [3]:
from crewai_tools import tool
@tool("Duck_Duck_Go_Search")
def ddgsearch(question: str) -> str:
    """Clear description for what this tool is useful for, your agent will need this information to use it."""
    # Function logic here
    return DuckDuckGoSearchRun().run(question)

In [4]:
# What is it that needs to get done?
research_topic = "Differences between open source large language models and closed source large language models."

In [5]:
researcher = Agent(
  role='Researcher',
  goal='Search the internet for information about {research_topic}',
  backstory="""
  You are a researcher. Using the information in the task, you find out some of the most popular facts about the topic along with some of the trending aspects.
  You provide a lot of information thereby allowing a choice in the content selected for the final blog.
  """,
  verbose=True,
  allow_delegation=False,
  # tools=[DuckDuckGoSearchRun()],
  tools=[ddgsearch],
  llm=ollama_llm
)

In [6]:
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on a set of information provided by the researcher.',
  backstory="""You are a writer known for your humorous but informative way of explaining. 
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=ollama_llm
)

In [7]:
task1 = Task(
  agent=researcher,
  description=research_topic,
  expected_output="A complete analysis of the {research_topic}, presented as a report.",
)

In [8]:
task2 = Task(
  agent=writer,
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant facts and differences between open-source LLMs and closed-source LLMs.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound simple, and avoid complex words so it doesn't sound AI generated.""",
  expected_output="""A article containing a minimum of 4 paragraphs. contains no more than 2500 words but no less than 1500 words.
  The final output will not contain any extra fluff like "Paragraph 1:" or any action the writer should do.""",
)

In [9]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 for different logging levels
)

In [10]:
# Get your crew to work!
result = crew.kickoff()

 [2024-06-24 07:55:32][DEBUG]: == Working Agent: Researcher
 [2024-06-24 07:55:32][INFO]: == Starting Task: Differences between open source large language models and closed source large language models.


> Entering new CrewAgentExecutor chain...
Action: Duck_Duck_Go_Search
Action Input: { "question": 'What are the differences between open source large language models and closed source large language models?' } 

Let's dig in to understand the difference between Open-Source LLM and Closed Source LLM. ... Open-source large language models, such as the ones offered by Meta AI, provide a foundational AI technology that can analyze and generate human-like text by learning from vast datasets consisting of various written materials. Here are some of the major types: 1. General-purpose LLMs. These models are trained on large amounts of general web text and aim to be useful for a wide range of tasks. Examples include: GPT-3 ... Large language models (LLMs) are a class of machine learning model

In [11]:
result

'Open-source and closed-source large language models (LLMs) differ primarily in terms of innovation, cost structure, customizability, collaboration, and accessibility. Open-source LLMs prioritize a transparent, customizable, and collaborative development approach with lower initial costs but potentially higher operational expenses. They are developed by various research teams and generally more accessible to smaller organizations and individual researchers. Closed-source LLMs emphasize innovative capabilities and structured support systems, often resulting from substantial investments in research and development by large corporations. These models provide a more polished user experience but may have limited customization options due to proprietary constraints.'

In [12]:
with open("Output.txt", "w") as text_file:
    print(result, file=text_file)